In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15072314719848832924
 xla_global_id: -1]

In [2]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [3]:
import numpy as np
import pandas as pd
import data_als
import data_prepro

In [4]:
train,test = data_prepro.loadData('datasets/data_tr_city.csv','datasets/data_ts_city.csv')

In [5]:
train = data_prepro.outlierDataToNan(train)
test = data_prepro.outlierDataToNan(test)

In [6]:
train = data_prepro.fillnaBehind(train)
test = data_prepro.fillnaBehind(test)

Index([ 1748,  1982,  2127,  3468,  3485,  3491,  3513,  3533,  3542,  3554,
        3593,  3607, 10208, 11995, 12519, 12716, 12717, 13187, 13234, 17558,
       26078, 26079, 27253, 27254],
      dtype='int64')
datetime               0
구미 혁신도시배수지 유출유량 적산차    0
year                   0
month                  0
dtype: int64
Index([1502], dtype='int64')
datetime               0
구미 혁신도시배수지 유출유량 적산차    0
year                   0
month                  0
dtype: int64


In [7]:
df = train

In [8]:
df['구미 혁신도시배수지 유출유량 적산차'].to_numpy()

array([138., 237., 128., ..., 335., 141., 112.])

In [9]:
X,Y = data_prepro.XDataToXAndYSeq(df)

In [10]:
X_test,Y_test = data_prepro.XDataToXAndYSeq(test)

In [11]:
Y.shape

(35038, 1)

In [12]:
Y

array([[113.],
       [ 51.],
       [ 10.],
       ...,
       [335.],
       [141.],
       [112.]])

In [13]:
X.shape

(35038, 24)

In [14]:
X

array([[138., 237., 128., ..., 152., 197., 175.],
       [237., 128.,  14., ..., 197., 175., 266.],
       [128.,  14.,  11., ..., 175., 266., 113.],
       ...,
       [305., 362., 207., ..., 473., 305., 328.],
       [362., 207., 133., ..., 305., 328., 347.],
       [207., 133., 153., ..., 328., 347., 335.]])

In [15]:
X = X.reshape((-1,24,1))

In [16]:
from tensorflow import keras

In [17]:
model = keras.Sequential()

model.add(keras.layers.GRU(256,input_shape=(24,1),return_sequences=True,activation='tanh'))
model.add(keras.layers.GRU(256,return_sequences=True,activation='tanh'))
model.add(keras.layers.GRU(256,return_sequences=True,activation='tanh'))
model.add(keras.layers.Conv1D(512,3,activation='relu',padding='SAME'))
model.add(keras.layers.Conv1D(512,3,activation='relu',padding='SAME'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024,activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(1))

In [18]:
model.compile(loss='mae',optimizer='adam',metrics=['accuracy'])

In [19]:
model.fit(X,Y,epochs=100,batch_size=1024,validation_data=(X_test, Y_test))

Epoch 1/100
35/35 [==============================] - 190s 5s/step - loss: 156.3753 - accuracy: 3.9957e-04 - val_loss: 126.3840 - val_accuracy: 0.0000e+00
Epoch 2/100
35/35 [==============================] - 187s 5s/step - loss: 103.3276 - accuracy: 5.9935e-04 - val_loss: 102.0698 - val_accuracy: 0.0000e+00
Epoch 3/100
35/35 [==============================] - 180s 5s/step - loss: 100.3351 - accuracy: 7.4205e-04 - val_loss: 101.9493 - val_accuracy: 0.0000e+00
Epoch 4/100
35/35 [==============================] - 188s 5s/step - loss: 99.8346 - accuracy: 0.0012 - val_loss: 96.6265 - val_accuracy: 0.0000e+00
Epoch 5/100
35/35 [==============================] - 188s 5s/step - loss: 98.7920 - accuracy: 0.0016 - val_loss: 102.2279 - val_accuracy: 0.0000e+00
Epoch 6/100
35/35 [==============================] - 191s 5s/step - loss: 98.0033 - accuracy: 0.0021 - val_loss: 97.7915 - val_accuracy: 0.0000e+00
Epoch 7/100
35/35 [==============================] - 191s 5s/step - loss: 97.2963 - accuracy:

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test,Y_test)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
263/263 [==============================] - 28s 100ms/step - loss: 104.7026 - accuracy: 0.0000e+00


[104.70259094238281, 0.0]

In [ ]:
model.save("100epochs_CPU.h5")